# Imports

In [1]:
import geopandas as gpd
import pandas as pd
from shapely import Point
from tqdm.autonotebook import tqdm

C:\Users\Jakub\AppData\Local\Temp\ipykernel_56712\2431355484.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Data Loading

In [2]:
df_porto_taxi = pd.read_parquet("data/porto_taxi.parquet")

In [3]:
df_porto_taxi.head()

,timestamp,trip_id,call_type,origin_call,origin_stand,taxi_id,day_type,speed,longitude,latitude
0,2013-07-01 02:04:44,1372637084620000285,C,NaN,NaN,20000285,A,9.733089,-8.585685,41.148576
1,2013-07-01 02:04:59,1372637084620000285,C,NaN,NaN,20000285,A,9.733089,-8.585766,41.148936
2,2013-07-01 02:05:14,1372637084620000285,C,NaN,NaN,20000285,A,12.120171,-8.586360,41.148864
3,2013-07-01 02:05:29,1372637084620000285,C,NaN,NaN,20000285,A,23.865080,-8.586243,41.147973
4,2013-07-01 02:05:44,1372637084620000285,C,NaN,NaN,20000285,A,24.176693,-8.587170,41.147397


In [4]:
df_porto_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2545947 entries, 0 to 2545946
Data columns (total 10 columns):
 #   Column        Dtype         
---  ------        -----         
 0   timestamp     datetime64[ns]
 1   trip_id       int64         
 2   call_type     object        
 3   origin_call   float64       
 4   origin_stand  float64       
 5   taxi_id       int64         
 6   day_type      object        
 7   speed         float64       
 8   longitude     float64       
 9   latitude      float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(2)
memory usage: 194.2+ MB


# Data Processing

In [5]:
tqdm.pandas()
df_porto_taxi["geometry"] = df_porto_taxi.progress_apply(
    lambda row: Point(row["longitude"], row["latitude"]), axis=1
)

  0%|          | 0/2545947 [00:00<?, ?it/s]

In [6]:
df_porto_taxi.drop(["longitude", "latitude"], axis=1, inplace=True)

In [7]:
gdf_porto_taxi = gpd.GeoDataFrame(
    data=df_porto_taxi, geometry="geometry", crs="EPSG:4326"
)

In [8]:
gdf_porto_taxi.head()

,timestamp,trip_id,call_type,origin_call,origin_stand,taxi_id,day_type,speed,geometry
0,2013-07-01 02:04:44,1372637084620000285,C,NaN,NaN,20000285,A,9.733089,POINT (-8.58568 41.14858)
1,2013-07-01 02:04:59,1372637084620000285,C,NaN,NaN,20000285,A,9.733089,POINT (-8.58577 41.14894)
2,2013-07-01 02:05:14,1372637084620000285,C,NaN,NaN,20000285,A,12.120171,POINT (-8.58636 41.14886)
3,2013-07-01 02:05:29,1372637084620000285,C,NaN,NaN,20000285,A,23.865080,POINT (-8.58624 41.14797)
4,2013-07-01 02:05:44,1372637084620000285,C,NaN,NaN,20000285,A,24.176693,POINT (-8.58717 41.14740)
